### Exercises:

Exercise: Suppose you have two coins in a box. 

- One is a normal coin with heads on one side and tails on the other, and one is a trick coin with heads on both sides. 
- You choose a coin at random and see that one of the sides is heads. 
    - Random is our prior; our data is seeing heads
- What is the probability that you chose the trick coin? 
    - Probability is `2/3`

In [4]:
import pandas as pd
from fractions import Fraction

def update(table):
    """Compute the posterior probabilities."""
    
    # solve unnormed posterior
    table['unnorm'] = table['prior'] * table['likelihood']
    
    # solve for probability of this data
    prob_data = table['unnorm'].sum()
    
    # build normed posterior 
    table['posterior'] = table['unnorm'] / prob_data
    
    return table

In [5]:
coin_table = pd.DataFrame(index=['normal', 'trick'])

# prior 1/2
coin_table['prior'] = Fraction(1, 2)

# likelihood is based on seeing coin
coin_table['likelihood'] = Fraction(1, 2), 1
                                                    
# let's do our update & print
coin_table = update(coin_table)

print(coin_table)

       prior likelihood unnorm posterior
normal   1/2        1/2    1/4       1/3
trick    1/2          1    1/2       2/3


Exercise: Suppose you meet someone and learn that they have two children. You ask if either child is a girl and they say yes. What is the probability that both children are girls?

Note: 
- I got this wrong originally because I set my likelihood as below for `GB` and `BG` to 0.5. 
- The part that threw me off was the "either" language - unlike the coin, the person sharing information knows both genders so if its `GB` there is a `1` likelihood. 

In [9]:
# wrong approach
kid_table = pd.DataFrame(index=['GG', 'GB', 'BG', 'BB'])

# prior 1/4 (we are told to assume this)
kid_table['prior'] = Fraction(1, 4)

# likelihood is based on child
kid_table['likelihood'] = 1, Fraction(1,2), Fraction(1,2), 0
                                                    
# let's do our update & print
kid_table = update(kid_table)

print(kid_table)

   prior likelihood unnorm posterior
GG   1/4          1    1/4       1/2
GB   1/4        1/2    1/8       1/4
BG   1/4        1/2    1/8       1/4
BB   1/4          0      0         0


In [10]:
# correct approach
kid_table = pd.DataFrame(index=['GG', 'GB', 'BG', 'BB'])

# prior 1/4 (we are told to assume this)
kid_table['prior'] = Fraction(1, 4)

# likelihood is based on either child
kid_table['likelihood'] = 1, 1, 1, 0
                                                    
# let's do our update & print
kid_table = update(kid_table)

print(kid_table)

   prior  likelihood unnorm posterior
GG   1/4           1    1/4       1/3
GB   1/4           1    1/4       1/3
BG   1/4           1    1/4       1/3
BB   1/4           0      0         0


Exercise: There are many variations of the Monty Hall problem.
For example, suppose Monty always chooses Door 2 if he can, and only chooses Door 3 if he has to (because the car is behind Door 2).

If you choose Door 1 and Monty opens Door 2, what is the probability the car is behind Door 3?
- This means 1 or 3 is correct
- P(Door 2 | Car Door 1) = 1 -> Monty always opens door 2 if car behind door 1
- P(Door 2 | Car Door 2) = 0 -> Monty would have opened Door 3
- P(Door 2 | Car Door 3) = 1 -> Monty always opens door 2 if car behind door 3

If you choose Door 1 and Monty opens Door 3, what is the probability the car is behind Door 2?
- This means the car is behind 2
- P(Door 3 | Car Door 1) = 0 -> If behind Door 1 monty would open door 2, not 3. 
- P(Door 3 | Car Door 2) = 1 -> If behind Door 2 monty opens door 3
- P(Door 3 | Car Door 3) = 0 -> Monty would have opened door 2 if car behind door 3.

In [15]:
# Monty Hall - P1
monty_one = pd.DataFrame(index=[1,2,3])

# prior: 1/3
monty_one['prior'] = Fraction(1, 3)

# likelihood is a bit trickier here:
# door 1 - he always opens door 3
# door 2 is 0 since it is opened
# door 3 - he always opens
monty_one['likelihood'] = 1, 0, 1
                                                    
# let's do our update & print
monty_one = update(monty_one)

print(monty_one)

  prior  likelihood unnorm posterior
1   1/3           1    1/3       1/2
2   1/3           0      0         0
3   1/3           1    1/3       1/2


In [16]:
# Monty Hall - P2
monty_two = pd.DataFrame(index=[1,2,3])

# prior: 1/3
monty_two['prior'] = Fraction(1, 3)

# likelihood
monty_two['likelihood'] = 0, 1, 0
                                                    
# let's do our update & print
monty_two = update(monty_two)

print(monty_two)

  prior  likelihood unnorm posterior
1   1/3           0      0         0
2   1/3           1    1/3         1
3   1/3           0      0         0


Exercise: M&M’s are small candy-coated chocolates that come in a variety of colors.
Mars, Inc., which makes M&M’s, changes the mixture of colors from time to time. In 1995, they introduced blue M&M’s.

In 1994, the color mix in a bag of plain M&M’s was 30% Brown, 20% Yellow, 20% Red, 10% Green, 10% Orange, 10% Tan.

In 1996, it was 24% Blue , 20% Green, 16% Orange, 14% Yellow, 13% Red, 13% Brown.

Suppose a friend of mine has two bags of M&M’s, and he tells me that one is from 1994 and one from 1996. He won’t tell me which is which, but he gives me one M&M from each bag. One is yellow and one is green. What is the probability that the yellow one came from the 1994 bag?

Hint: The trick to this question is to define the hypotheses and the data carefully.

In [20]:
# MM
mm = pd.DataFrame(index=['1994', '1996'])

# prior: 1/2
mm['prior'] = Fraction(1, 2)

# likelihood of yellow from bag, green from other bag?
mm['likelihood'] = 0.2 * 0.2, .14 * 0.1
                                                    
# let's do our update & print
mm = update(mm)

print(mm)

     prior  likelihood unnorm posterior
1994   1/2       0.040   0.02  0.740741
1996   1/2       0.014  0.007  0.259259


#### Breakdown of Above:

The problem is slightly different as we need to account for two pieces of data. 
- We still assume each hypothesis has equal likelihood
- The hypotheses account for more than just yellow:
    - 1994: Y from 1994, G from 1996
    - 1996: Y from 1996, B from 1994
- Likelihood:
    - 1 M&M is pulled from each bag, so each pull in independent
    - This means we can just multiply each of the likelihods as done above. 
    
- Intuitively, the result makes sense:
    - P(Y 94) > P(Y 96) AND P(G 96) > P(G 94)
    